In [30]:
import pandas as pd
import jsonpickle
import numpy as np
import matplotlib.pyplot as plt
import os, os.path
import dateutil.parser
from datetime import datetime

Constants

In [31]:
FILE_AS_ROOT = False
SELF_PATH = os.getcwd()#os.path.dirname(os.path.abspath(__file__))
PATH_TO_ROOT = os.path.join(SELF_PATH,'../../../../../../' if FILE_AS_ROOT else '../')
PATH_TO_JSON_STATE = os.path.join(PATH_TO_ROOT,'latest_state.json')

Given path to a pickle state,will return pickle object

In [32]:
def get_pickle(path_to_file):
    with open(path_to_file, 'r') as f:
        return jsonpickle.decode(f.read())

Given a pickgle object will return how many incomplete, complete images have been tagged along with unix epoch time and date time stamp

In [33]:
def how_many_tagged(pickle_file):
    
    done_tagging_count=len(pickle_file.finished_tagged_queue)
    tagged_but_not_done_count=0
    
    for image in pickle_file.pending_images_queue:
        if len(image.get_taggers()) > 0:
            tagged_but_not_done_count+=1
    
    return pd.DataFrame([{ 
        'not_done':tagged_but_not_done_count,
        'done':done_tagging_count,
        'tagged_ratio': tagged_but_not_done_count/(tagged_but_not_done_count+done_tagging_count)
     }])

In [44]:
pickle = get_pickle(PATH_TO_JSON_STATE)

,not_done,done,tagged_ratio
0,117,248,0.320548


In [43]:
# for image in pickle.finished_tagged_queue:
#     times_stop = image.stats_tagging_stop
#     keys = list(times_stop.keys())
#     last_tagger = keys[-1]
#     last_time = times_stop.get(last_tagger)
#     print(datetime.fromtimestamp(last_time))

In [45]:
finished_images = pickle.finished_tagged_queue

In [117]:
image_tag_df = pd.DataFrame(
    columns=['image_id', 'tagger','development','impact','notes','washover']
)

for image in finished_images:

    tagger_list = list(image.get_taggers())
    image_id = image.get_rel_path()

    for tagger in tagger_list:

        list_of_tags = (image.get_tags(tagger))
        
        df_row ={
            'image_id':image_id,
            'tagger':tagger,
            
        }

        df_row.update(list_of_tags)
        
        row_df = pd.DataFrame([df_row]) 
        #image_tag_df.merge(row_df)
        image_tag_df = pd.concat([image_tag_df,row_df],sort=True)
        
image_tag_df.head()

,development,image_id,impact,notes,ocean,tagger,terrain_inland,terrain_marsh,terrain_river,terrain_sandy_coastline,terrain_undefined,washover
0,0,jpgs/S26047238.jpg,0,,NaN,google-oauth2|112971301179841927329,NaN,NaN,NaN,NaN,True,0
0,0,jpgs/S26047238.jpg,0,,NaN,google-oauth2|101534126929210992885,NaN,NaN,NaN,NaN,True,0
0,1,jpgs/P26057146.jpg,2,,NaN,google-oauth2|112971301179841927329,NaN,NaN,NaN,True,NaN,0
0,1,jpgs/P26057146.jpg,2,,NaN,google-oauth2|101534126929210992885,NaN,NaN,NaN,True,NaN,0
0,1,jpgs/P26054308.jpg,2,,NaN,google-oauth2|112971301179841927329,NaN,NaN,NaN,True,NaN,0
